In [65]:
import pandas as pd
import os
from py2neo import Node, Graph, Relationship, NodeMatcher

In [66]:
graph = Graph("http://localhost:7474/", auth=("neo4j", "123"))

In [67]:
graph.delete_all()

In [68]:
df = pd.DataFrame(pd.read_excel('双碳政策-20220905.xls'))
attr = ['机构', '日期', '类型', '领域', '省份', '城市']
df_lst = [df.iloc[:, :3],df.iloc[:, [0, 3, 4]],df.iloc[:, [0, 5, 6]],df.iloc[:, [0, 7, 8]],df.iloc[:, [0, 9, 10]],df.iloc[:, [0, 11, 13]]]
triples= []
node_info = []
flag = True
for attr, df in zip(attr, df_lst):
    for i in range(len(df)):
        line = df.loc[i]
        line = list(dict(line).values())
        triples.append((attr, line))
        if flag:
            node_info.append(('policy', line[0]))
        node_info.append((line[1], line[2]))
    flag = False
df_lst = []

In [69]:
# 建立一个节点
def create_node(graph, label, attrs):
    n = "_.name=" + "\"" + attrs["name"] + "\""
    matcher = NodeMatcher(graph)
    # 查询是否已经存在，若存在则返回节点，否则返回None
    value = matcher.match(label).where(n).first()
    # 如果要创建的节点不存在则创建
    if value is None:
        node = Node(label, **attrs)
        n = graph.create(node)
        return n
    return None

In [70]:
for label, name in node_info:
    if type(label) != str or type(name) != str:
        continue
    create_node(graph, label, {'name': name})

In [71]:
# 建立两个节点之间的关系
def create_relationship(graph, label1, attrs1, label2, attrs2, r_name):
    value1 = match_node(graph, label1, attrs1)
    value2 = match_node(graph, label2, attrs2)
    if value1 is None or value2 is None:
        return False
    r = Relationship(value1, r_name, value2)
    graph.create(r)


# 查询节点
def match_node(graph, label, attrs):
    n = "_.name=" + "\"" + attrs["name"] + "\""
    matcher = NodeMatcher(graph)
    return matcher.match(label).where(n).first()

In [ ]:
for relation, (subject, verb, object) in triples:
    if type(verb) != str:
        continue
    create_relationship(graph, 'policy', {'name': subject}, verb, {'name': object}, relation)